In [ ]:
import pandas as pd
import numpy as np
import sys, os
import matplotlib.pyplot as plt
import skimage.io
# ceļu norādīt nevajag, ja pakotne ir instalēta
documentpath=os.getcwd()
libpath=os.path.abspath(os.path.join(documentpath, os.pardir))
sys.path.append(libpath)
from sudrabainiemakoni.cloudimage import CloudImage
from sudrabainiemakoni.cloudimage import CloudImagePair
from sudrabainiemakoni.cloudimage import WebMercatorImage
from sudrabainiemakoni import plots

In [ ]:
# katalogs ar piemēra failiem
sample_directory = 'SampleData'
# katalogs ar rezultātu failiem
results_directory = 'SampleResults'
# pirmais attēls
case1 = 'js_202106210100'
# otrais attēls
case2 = ''

In [ ]:
# izveidojam divus CloudImage tipa objektus, katru savam attēlam
# Ielasam no faila, pirms tam referencētus attēlus
cldim1 = CloudImage.load(f'{results_directory}/{case1}.proj')
cldim2 = CloudImage.load(f'{results_directory}/{case2}.proj')
print(cldim1)
print(cldim2)

Veidosim neatkarīgas abu sudrabaino mākoņu projicēto attēlu rindas dažādos augstumos
Apvienosim attēlus ar dažādām metodēm, šādu attēlu rindu aplūkošana ļauj izprast kā atrast katra mākoņu punkta augstumu - vizuāli vērojot abu attēlu sakrītību 

In [ ]:
# augstumu intervāli, kilometros 70-94.5, ik pa 0.5 km
heights = np.arange(70, 95, 0.5)

### Projicējam uz dažādiem augstumiem divus attēlus

In [ ]:
# projicēšanas apgabals un izšķirtspēja
lonmin, lonmax, latmin, latmax, horizontal_resolution_km = 17, 31, 57.6, 62, 0.5

In [ ]:
# sagatavojam projicēšanas objektus
webmerc1 = WebMercatorImage(cldim1, lonmin, lonmax, latmin, latmax, horizontal_resolution_km)
webmerc2 = WebMercatorImage(cldim2, lonmin, lonmax, latmin, latmax, horizontal_resolution_km)

In [ ]:
# noglabāsim projicētos attēlus TIF failos
outdir = f'{results_directory}/projZ'
if not os.path.exists(outdir):
    os.makedirs(outdir)
if not os.path.exists(f'{outdir}/{case1}'):
    os.makedirs(f'{outdir}/{case1}')
if not os.path.exists(f'{outdir}/{case2}'):
    os.makedirs(f'{outdir}/{case2}')

In [ ]:

for height in heights:
    print('Augstums', height)
    sh = f"{height:.1f}".replace('.','_')
    fn1 = f'{outdir}/{case1}/p_{sh}.jpg'
    fn2 = f'{outdir}/{case2}/p_{sh}.jpg'
    webmerc1.prepare_reproject_from_camera(height)
    projected_image1=webmerc1.Fill_projectedImage()
    skimage.io.imsave(fn1, projected_image1)
    webmerc2.prepare_reproject_from_camera(height)
    projected_image2=webmerc2.Fill_projectedImage()
    skimage.io.imsave(fn2, projected_image2)

In [ ]:
from sudrabainiemakoni import utils

In [ ]:
for subdir in ['mean','diff','bicolor']:
    if not os.path.exists(f'{outdir}/{subdir}'):
        os.makedirs(f'{outdir}/{subdir}')

In [ ]:
# izveidojam dažādu krāsojumu dubultos attēlus
for height in heights:
    print('Augstums', height)
    sh = f"{height:.1f}".replace('.','_')
    fn1 = f'{outdir}/{case1}/p_{sh}.jpg'
    fn2 = f'{outdir}/{case2}/p_{sh}.jpg'
    im1=skimage.io.imread(fn1)
    im2=skimage.io.imread(fn2)
    img_mean, img_diff, _, img_bicolor = utils.getAverageImages([im1,im2])
    utils.writeWithText(img_mean, height,f'{outdir}/mean/{case1}_{case2}_{sh}.jpg')
    utils.writeWithText(img_diff, height,f'{outdir}/diff/{case1}_{case2}_{sh}.jpg')
    utils.writeWithText(img_bicolor[(0,1)], height,f'{outdir}/bicolor/{case1}_{case2}_{sh}.jpg')


### Projicējam uz otro attēlu pirmo attēlu caur dažādiem augstumiem (dubultprojicēšana)

In [ ]:
imagepair = CloudImagePair(cldim1, cldim2)

In [ ]:
outdir = f'{results_directory}/projZ/proj12'
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [ ]:
#  noglabājam attēlu, kur pirmās kameras attēls projicēts uz otrās kameras pozīciju caur fiksētu augstumu
for height in heights:
    sh = f"{height:.1f}".replace('.','_')
    imagepair.reproject.prepare_reproject_1_2(height)
    projected_image=imagepair.reproject.Fill_projectedImage()
    print(f'Double reprojected image {case1} at height {height} km ')
    skimage.io.imsave(f"{outdir}\\proj_{case1}_to_{case2}_{sh}.jpg", projected_image)

In [ ]:
for subdir in ['mean','diff','bicolor']:
    if not os.path.exists(f'{outdir}/{subdir}'):
        os.makedirs(f'{outdir}/{subdir}')

In [ ]:
# izveidojam dažādu krāsojumu dubultos attēlus
for height in heights:
    imgarr=[cldim2.imagearray]
    sh = f"{height:.1f}".replace('.','_')
    img_=skimage.io.imread(f"{outdir}\\proj_{case1}_to_{case2}_{sh}.jpg")
    imgarr.append(img_)
    imgarr=np.array(imgarr)
    img_mean, img_diff, _ , img_bicolor = utils.getAverageImages(imgarr)
    utils.writeWithText(img_mean, height, f'{outdir}/mean/{case1}_{case2}_{sh}.jpg')
    utils.writeWithText(img_diff, height, f'{outdir}/diff/{case1}_{case2}_{sh}.jpg')
    utils.writeWithText(img_bicolor[(0,1)], height,f'{outdir}/bicolor/{case1}_{case2}_{sh}.jpg')
